In [63]:
import json
import os
import sys
import inspect

from src.data import make_dataset
from src.features import build_features
from src.features import metrics_analysis

In [64]:
## Creating paths to store temp and out data ##
if not os.path.exists("data/temp"):
    os.makedirs("data/temp")
if not os.path.exists("data/out"):
    os.makedirs("data/out")

In [65]:
from qiime2.plugins import feature_table
from qiime2 import Artifact
from qiime2.plugins.sample_classifier.pipelines import classify_samples
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2 import Metadata
from qiime2.plugins.diversity.pipelines import core_metrics
from qiime2.plugins.diversity.pipelines import core_metrics_phylogenetic
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.diversity.methods import umap


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import biom
import skbio

import seaborn as sns
# %matplotlib inline 

## Loading Data

In [66]:
## Obtaining file paths
with open("config/data-params.json") as fh:
    file_paths = json.load(fh)

In [67]:
table = make_dataset.read_feature_table(file_paths["feature_table_path"])
tree = make_dataset.read_tree_table(file_paths["tree_path"])

## Load Metadata

In [68]:
qiime_metadata = Metadata.load("data/temp/final_metadata.tsv") #Cleaned metadata
metadata_df = qiime_metadata.to_dataframe()

## Dividing Metadata based on Hispanic Origin

We are going to divide and group up our data based on the hispanic origin variable to see if there are any patterns we can discover from the groupings

Metadata Description: (1=Dominican, 2=Central American, 3=Cuban, 4=Mexican, 5=Puerto Rican, 6=South American, 7=More than one heritage, 8=other)

In [69]:
#Describe the hispanic_origin column values
metadata_df['hispanic_origin'].value_counts()

4.0    863
5.0    220
3.0    216
2.0    170
1.0    123
6.0    117
7.0     28
8.0     10
Name: hispanic_origin, dtype: int64

### Select hispanic origin samples of interest
For now, look into the top 3 hispanic origin categories: Mexican, Puerto Rican, and Cuban origin

In [70]:
metadata_df_m = metadata_df.loc[(metadata_df['hispanic_origin'] == 4)] #Mexico
metadata_df_pr = metadata_df.loc[(metadata_df['hispanic_origin'] == 5)] #Puerto Rico
metadata_df_c = metadata_df.loc[(metadata_df['hispanic_origin'] == 3)] #Cuba    


In [71]:

metadata_df_m.to_csv('data/temp/final_metadata_m.tsv', sep="\t")
metadata_df_pr.to_csv('data/temp/final_metadata_pr.tsv', sep="\t")
metadata_df_c.to_csv('data/temp/final_metadata_c.tsv', sep="\t")

In [72]:
metadata_m = Metadata.load("data/temp/final_metadata_m.tsv") 
metadata_pr = Metadata.load("data/temp/final_metadata_pr.tsv") 
metadata_c = Metadata.load("data/temp/final_metadata_c.tsv") 

Create the 3 filtered feature tables for the 3 hispanic origins

In [73]:
feature_table_m = filter_samples(table, metadata = metadata_m).filtered_table
df_feature_table_m = feature_table_m.view(pd.DataFrame)

feature_table_pr = filter_samples(table, metadata = metadata_pr).filtered_table
df_feature_table_pr = feature_table_pr.view(pd.DataFrame)

feature_table_c = filter_samples(table, metadata = metadata_c).filtered_table
df_feature_table_c = feature_table_c.view(pd.DataFrame)

In [74]:
print(df_feature_table_m.shape)
print(df_feature_table_pr.shape)
print(df_feature_table_c.shape)

(863, 31600)
(220, 9465)
(216, 9567)


## Feature Table Exploration

In [75]:
#Filter feature table based on cleaned metadata
updated_feature_table = filter_samples(table, metadata = qiime_metadata).filtered_table
biom_table = updated_feature_table.view(biom.Table)
biom_table #around 4000 rows was removed

57181 x 1747 <class 'biom.table.Table'> with 342969 nonzero entries (0% dense)

In [76]:
summary = summarize(updated_feature_table, qiime_metadata)
summary.visualization.save('data/out/ft_summary')

'data/out/ft_summary.qzv'

From the summary visualizations and statistics, we see that most features only appear in less than 3 samples, therefore we are going to drop the features that appear less than 3 times in order to reduce noise.

In [77]:
feat_table = updated_feature_table.view(pd.DataFrame)
feat_table.shape

(1747, 57181)

In [78]:
#Drop the feature columns that appear in less than 3 samples
feat_table_3 = feat_table[feat_table.columns[((feat_table > 0).sum() > 3)]]

#Import DataFrame back into FeatureTable artifact and export the summary
cleaned_feature_table = Artifact.import_data("FeatureTable[Frequency]", feat_table_3)

summary_cleaned = summarize(cleaned_feature_table, qiime_metadata)
summary_cleaned.visualization.save('data/out/ft_summary_3')

'data/out/ft_summary_3.qzv'

In [79]:
feat_table_3.shape

(1747, 3985)

### Perform Feature Table Cleaning on hispanic_origin groups

In [80]:
#Drop the feature columns that appear in less than 3 samples
df_feature_table_m = df_feature_table_m[df_feature_table_m.columns[((df_feature_table_m > 0).sum() > 3)]]
df_feature_table_pr = df_feature_table_pr[df_feature_table_pr.columns[((df_feature_table_pr > 0).sum() > 3)]]
df_feature_table_c = df_feature_table_c[df_feature_table_c.columns[((df_feature_table_c > 0).sum() > 3)]]

#Import DataFrame back into FeatureTable artifact and export the summary
cleaned_feature_table_m = Artifact.import_data("FeatureTable[Frequency]", df_feature_table_m)
cleaned_feature_table_pr = Artifact.import_data("FeatureTable[Frequency]", df_feature_table_pr)
cleaned_feature_table_c = Artifact.import_data("FeatureTable[Frequency]", df_feature_table_c)


In [81]:
#Create the summary visualizations and export the files to view

summary_m = summarize(cleaned_feature_table_m, metadata_m)
summary_m.visualization.save('data/out/ft_summary_m')

summary_pr = summarize(cleaned_feature_table_pr, metadata_pr)
summary_pr.visualization.save('data/out/ft_summary_pr')

summary_c = summarize(cleaned_feature_table_c, metadata_c)
summary_c.visualization.save('data/out/ft_summary_c')

'data/out/ft_summary_c.qzv'

# Feature Table Metrics Analysis

First figure out the feature table rarefication, save the plots generated by Qiime2 core_metrics, then create the following:
1. Distance matrices: Unifrac distance matrix, Jaccard distance matrix, Bray Curtis distance matrix
2. PCOA plots with the different distance matrices
    - save the plots for visualization
    
    
3. UMAP plots with the different distance matrices
    - save the plots for visualization
    
    
4. Finally follow up with a statistical test or regression
    - Ex) PERMANOVA test on PCOA results
    - Ex) Use the reduced dimension embeddings to feed into the regression model

With the information we gained from the summary, we decided to rarefy the table with a sampling depth of _7930_ we retained _12,489,750 (52.48%) features in 1575 (90.00%) samples_ at the specifed sampling depth. We made this decision to maximize the amount of features while preserving the amount of samples in our data.

In [82]:
# # Create the feature table metrics object

# feat_table = cleaned_feature_table
# depth = 7930
# metadata = qiime_metadata
# feature_table_metrics = metrics_analysis.extract_core_metrics(feat_table, depth, metadata)


In [83]:
# Create the feature table metrics object with tree
feat_table = cleaned_feature_table
depth = 7930
metadata = qiime_metadata
feature_table_metrics_phy = core_metrics_phylogenetic(feat_table, tree, depth, metadata)

feature_table_metrics_phy

/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.05834027375794446 and the largest is 34.06313182852818.
  warn(
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitu

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/renaldy/data/38c483c9-3966-476f-b636-7ba8bbe4eb62/data/feature-table.biom -t /tmp/qiime2/renaldy/data/89eb2cfa-9e8c-462a-a9a7-93e7812fa767/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-2wwhzx4b

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/renaldy/data/38c483c9-3966-476f-b636-7ba8bbe4eb62/data/feature-table.biom -t /tmp/qiime2/renaldy/data/89eb2cfa-9e8c-462a-a9a7-93e7812fa767/data/tree.nwk -m unweighted -o /tmp/q2-LSMatFormat-4euho9ut

Running external command line application. This may print messages to stdout and/or stderr.
The c

/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.09421391161380219 and the largest is 37.90000641414089.
  warn(
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section 

Results (name = value)
----------------------------------------------------------------------------------------------------------------------
rarefied_table                     = <artifact: FeatureTable[Frequency] uuid: c6bd94f0-bfa4-444d-bd37-679c0b622152>
faith_pd_vector                    = <artifact: SampleData[AlphaDiversity] uuid: 52520710-97f1-4060-ad64-27732ce2299f>
observed_features_vector           = <artifact: SampleData[AlphaDiversity] uuid: 2801fbfb-eb64-4d04-9a6c-2ad972aaaa7b>
shannon_vector                     = <artifact: SampleData[AlphaDiversity] uuid: 10d4f644-f932-4765-9091-7647b2cca9ef>
evenness_vector                    = <artifact: SampleData[AlphaDiversity] uuid: 18e15796-d8a3-4953-bec9-c338b04a11e2>
unweighted_unifrac_distance_matrix = <artifact: DistanceMatrix uuid: 4c9093d5-a1d3-4058-bc49-02e974876d19>
weighted_unifrac_distance_matrix   = <artifact: DistanceMatrix uuid: 8e425c42-6413-45b6-8157-8b8d8f13df39>
jaccard_distance_matrix            = <artifact: Dist

In [84]:
# Create the feature table metrics object with tree for Mexico
feat_table = cleaned_feature_table_m
depth = 7000
metadata = metadata_m
feature_table_metrics_m = core_metrics_phylogenetic(feat_table, tree, depth, metadata)

/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.04936025372116822 and the largest is 18.342019707110428.
  warn(
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnit

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/renaldy/data/1d1192cd-3e6c-4f6a-b346-4d0d250a97fd/data/feature-table.biom -t /tmp/qiime2/renaldy/data/89eb2cfa-9e8c-462a-a9a7-93e7812fa767/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-5me1g1k4

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/renaldy/data/1d1192cd-3e6c-4f6a-b346-4d0d250a97fd/data/feature-table.biom -t /tmp/qiime2/renaldy/data/89eb2cfa-9e8c-462a-a9a7-93e7812fa767/data/tree.nwk -m unweighted -o /tmp/q2-LSMatFormat-0r329_08

Running external command line application. This may print messages to stdout and/or stderr.
The c

/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.06728722851041266 and the largest is 20.67073825634418.
  warn(
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section 

In [85]:
# Create the feature table metrics object with tree for Mexico
feat_table = cleaned_feature_table_pr
depth = 7000
metadata = metadata_pr
feature_table_metrics_pr = core_metrics_phylogenetic(feat_table, tree, depth, metadata)

/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.001854844314110109 and the largest is 5.074042599869747.
  warn(
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnit

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/renaldy/data/854d7816-31d5-41bb-9b92-272b96ef290a/data/feature-table.biom -t /tmp/qiime2/renaldy/data/89eb2cfa-9e8c-462a-a9a7-93e7812fa767/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-edpb3cit

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/renaldy/data/854d7816-31d5-41bb-9b92-272b96ef290a/data/feature-table.biom -t /tmp/qiime2/renaldy/data/89eb2cfa-9e8c-462a-a9a7-93e7812fa767/data/tree.nwk -m unweighted -o /tmp/q2-LSMatFormat-4xe_jgz1

Running external command line application. This may print messages to stdout and/or stderr.
The c

/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.041587443974489345 and the largest is 4.644538342298988.
  warn(
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section

In [86]:
# Create the feature table metrics object with tree for Mexico
feat_table = cleaned_feature_table_c
depth = 7000
metadata = metadata_c
feature_table_metrics_c = core_metrics_phylogenetic(feat_table, tree, depth, metadata)

/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.3044572224339407 and the largest is 17.485365785092966.
  warn(


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/renaldy/data/30ebdf98-8c5a-4cd3-906e-2e4e7cb6f2ee/data/feature-table.biom -t /tmp/qiime2/renaldy/data/89eb2cfa-9e8c-462a-a9a7-93e7812fa767/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-x_f6ul8a

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/renaldy/data/30ebdf98-8c5a-4cd3-906e-2e4e7cb6f2ee/data/feature-table.biom -t /tmp/qiime2/renaldy/data/89eb2cfa-9e8c-462a-a9a7-93e7812fa767/data/tree.nwk -m unweighted -o /tmp/q2-LSMatFormat-exe0a6vf

Running external command line application. This may print messages to stdout and/or stderr.
The c

/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.03375287984502647 and the largest is 5.4218673516432645.
  warn(
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section

In [87]:
feature_table_metrics_m.weighted_unifrac_emperor.save('data/out/w_unifrac_pcoa_m')
feature_table_metrics_c.weighted_unifrac_emperor.save('data/out/w_unifrac_pcoa_c')
feature_table_metrics_pr.weighted_unifrac_emperor.save('data/out/w_unifrac_pcoa_pr')

'data/out/w_unifrac_pcoa_pr.qzv'

In [88]:
#Exploration of the dataset
# Print rarefied table
rarefied_table = feature_table_metrics.rarefied_table
rarefied_df = rarefied_table.view(pd.DataFrame)
rarefied_df.shape #Rarefied table dropped some samples that are less than the n=7930 sampling depth

NameError: name 'feature_table_metrics' is not defined

In [ ]:
#Calculate Distance matrices
distance_matrices = metrics_analysis.extract_distance_matrices(feature_table_metrics)
jac_dis_matrix = distance_matrices[0]
bc_dis_matrix = distance_matrices[1]

(<artifact: DistanceMatrix uuid: a4110a9a-66d9-4502-a7b4-2b83f069231c>,
 <artifact: DistanceMatrix uuid: 4d96891d-81c6-419e-b18e-e16ff60cf05c>)

In [ ]:
#Calculate the PCOA matrices
pcoa_matrices = metrics_analysis.extract_pcoa_results(feature_table_metrics)
pcoa_matrices

(<artifact: PCoAResults uuid: 947398a7-d746-4a29-a56f-d5a053a9952c>,
 <artifact: PCoAResults uuid: e017dac0-fdb3-43c2-a56d-22365d255e76>)

In [ ]:
#Calculate the Emperor visualization and output
pcoa_emperor_plots = metrics_analysis.extract_pcoa_emperor_vis(feature_table_metrics)
pcoa_emperor_plots

(<visualization: Visualization uuid: 316ac20e-25fc-4203-af63-ecf93aa9a5a8>,
 <visualization: Visualization uuid: df780e27-f64b-4698-b748-c03af026898f>)

In [ ]:
def save_pcoa_outputs(metrics):
    jac_pcoa = metrics.jaccard_pcoa_results
    bc_pcoa = metrics.bray_curtis_pcoa_results
    
    jaccard_emperor = metrics.jaccard_emperor
    bray_curtis_emperor = metrics.bray_curtis_emperor
    
    jac_pcoa.save('data/out/jac_pcoa_matrix')
    bc_pcoa.save('data/out/bc_pcoa_matrix')
    
    jaccard_emperor.save('data/out/jac_pcoa_emp')
    bray_curtis_emperor.save('data/out/bc_pcoa_emp')
    
    return

save_pcoa_outputs(feature_table_metrics)

### UMAP Dimensionality Reduction
Perform some tasks using the UMAP artifact in Qiime2 as well as the Supervised UMAP from UMAP library

In [ ]:
#Code for UMAP using qiime2
for n in (2, 5, 10, 15, 20, 50):
    dim = 3
    umap_results = metrics_analysis.extract_umap_results(u_unifrac[0], dim, n)[0]
    umap_vis = metrics_analysis.extract_umap_vis(umap_results, metadata)[0]
    file_path = "data/out/umap_vis_u_unifrac_3_" + str(n)
    umap_vis.save(file_path)

/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/q2_diversity/_ordination.py:96: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  proportion_explained=pd.Series(None, index=axis_labels),
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/q2_diversity/_ordination.py:96: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  proportion_explained=pd.Series(None, index=axis_labels),
/home/renaldy/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/q2_diversity/_ordination.py:96: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  proportion_explained=pd.Series(None, index=axis_labels),
/home/renaldy/miniconda3/

### Supervised UMAP
First i'm going to try to use supervised UMAP with 'disease_type' as the target on samples that only have 1 disease. Then try to apply this to the whole dataset and see if we can still get good results.
Going to attempt to visualize in the 2D space first, then 3D space using emperor. 2D plots code will be taken from UMAP documentation.


In [104]:
#Filtering metadata to only get samples with 1 disease type, to remove disease ambiguity
filtered_metadata_df = metadata_df.drop('hispanic_origin', axis=1)

filtered_metadata_df = filtered_metadata_df[(filtered_metadata_df.sum(axis=1) == 1)] #We have around 300 samples left
filtered_metadata_df

,abdominal_obesity_ncep_v2,ckd_v2,diabetes2_v2,hypertension2_v2,precvd_v2,elevated_bp_selfmeds_v2,dyslipidemia_v2
sample_name,,,,,,,
11666.G0003A,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11666.G0009A,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11666.G0014A,0.0,1.0,0.0,0.0,0.0,0.0,0.0
11666.G0034A,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11666.G0037A,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
11666.G1696A,0.0,1.0,0.0,0.0,0.0,0.0,0.0
11666.G1716A,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11666.G1745A,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [105]:
#Convert binary columns into a single categorical target column for disease type
filtered_metadata_df["target_disease"] = filtered_metadata_df.idxmax(axis=1)

#Get the target disease and sample name,then create the metadata and filterd feature table
target_disease = filtered_metadata_df['target_disease']
target_disease.to_csv('data/temp/target_disease.tsv', sep="\t")
metadata_target_disease = Metadata.load("data/temp/target_disease.tsv") 
feature_table_target_disease = filter_samples(table, metadata = metadata_target_disease).filtered_table

In [107]:
#View as a DF to perform cleaning
feature_df_target_disease = feature_table_target_disease.view(pd.DataFrame)
feature_df_target_disease = feature_df_target_disease[feature_df_target_disease.columns[((feature_df_target_disease > 0).sum() > 3)]] #Dropping features that appear in less than 3 samples
feature_df_target_disease

,AACATAGGGGGCAAGCGTTGTCCGGAATCACTGGGCGTAAAGGGCGCGTAGGTGGTCTGTTAAGTCAGATGTGAAATGTAAGGGCTCAACCCTTAACGTGCATCTGATACTGGCAGACTTGAGTGCGGAAGAGGCAAGTGGAATTCCTAG,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAAACCATAGGCTCAACCTATGGATTGCTTTCAAAACTGCTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGG,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAAACCATGGGCTCAACCCATGAATTGCTTTCAAAACTGCTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGG,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAAACCATGGGCTCAACCCGTGAATTGCTTTCAAAACTGCTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGG,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAAACTATGGGCTCAACCCATAAATTGCTTTCAAAACTGCTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGG,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAAACTATGGGCTCAACTCATAAATTGCTTTCAAAACTGCTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGG,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAAACTATGGGCTTAACCCATAAATTGCTTTCAAAACTGCTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGG,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAATCCATGGGCTCAACCCGTGAATTGCTTTCAAAACTGCTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGG,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAATCCATGGGCTTAACCCGTGAATTGCTTTCAAAACTGCTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGG,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGGAGTGAAATCCATGGGCTCAACCCGTGAATTGCTTTCAAAACTGCTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGG,...,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGTGCGTAGGTGGTATGGCAAGTCAGAAGTGAAAGGCTGGGGCTCAACCCCGGGACTGCTTTTGAAACTGTCAAACTAGAGTACAGGAGAGGAAAGCGGAATTCCTAG,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGTGCGTAGGTGGTGAGACAAGTCTGAAGTGAAAATCCGGGGCTTAACCCCGGAACTGCTTTGGAAACTGCCTGACTAGAGTACAGGAGAGGTAAGTGGAATTCCTAG,TACGTATGTCCCGAGCGTTATCCGGATTTATTGGGCGTAAAGCGAGCGCAGACGGTTGATTAAGTCTGATGTGAAAGCCCGGAGCTCAACTCCGGAATGGCATTGGAAACTGGTTAACTTGAGTGTTGTAGAGGTAAGTGGAACTCCATG,TACGTATGTCGCAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCGTCTAGGCGGCTTAGTAAGTCTGATGTGAAAATGCGGGGCTCAACCCCGTATTGCGTTGGAAACTGCTAAACTAGAGTACTGGAGAGGTAGGCGGAACTACAAGT,TACGTATGTCGCAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCGTCTAGGCGGTTTAGTAAGTCTGATGTGAAAATGCGGGGCTCAACCCCGTATTGCGTTGGAAACTGCTAAACTAGAGTACTGGAGAGGTAGGCGGAACTACAAGT,TACGTATGTCGCAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCGTCTAGGCGGTTTGGTAAGTCTGATGTGAAAATGCGGGGCTCAACTCCGTATTGCGTTGGAAACTGCCAAACTAGAGTACTGGAGAGGTGGGCGGAACTACAAGT,TACGTATGTCGCGAGCGTTATCCGGAATTATTGGGCATAAAGGGCATCTAGGCGGATATACAAGTCAGGGGTGAAAACTTAGGGCTCAACTCAAAGCTTGCCTTTGAAACTGTATATCTAGAGTGCTGGAGAGGTGGACGGAACTACACG,TACGTATGTTCCAAGCGTTATCCGGATTTATTGGGCGTAAAGCGAGCGCAGACGGTTATTTAAGTCTGAAGTGAAAGCCCTCAGCTCAACTGAGGAATTGCTTTGGAAACTGGATGACTTGAGTGCAGTAGAGGAAAGTGGAACTCCATG,TACGTGGGTCGCAAGCGTTATCCGGAATCATTGGGCGTAAAGAGTGAGCAGGCGGTCCGGGAAGTCCGCGGTGAAATGCGGGGGCTCAACCCCCGCAGGCCGCGGATACTTCCGGTCTGGGGTGCGGGAGAGGCGGACGGAACTCCGCGG,TACGTGGGTCGCAAGCGTTATCCGGAATCATTGGGCGTAAAGAGTGAGCAGGCGGTCCGGGAAGTCCGCGGTGAAATGCGGGGGCTCAACCCCCGCAGGCCGCGGATACTTCCGGTCTGGGGTGCGGGAGAGGCGGACGGAACTCCGCGT
11666.G0394A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11666.G0156A,0.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11666.G0343A,0.0,0.0,0.0,0.0,84.0,0.0,0.0,0.0,0.0,0.0,...,465.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11666.G0103A,3.0,0.0,0.0,0.0,44.0,0.0,0.0,2.0,0.0,0.0,...,768.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
11666.G0366A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11666.G1477A,0.0,0.0,0.0,0.0,45.0,0.0,9.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11666.G1494A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11666.G1648A,0.0,0.0,0.0,4.0,4.0,0.0,0.0,1.0,0.0,0.0,...,134.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11666.G1650A,2.0,0.0,0.0,4.0,14.0,0.0,0.0,0.0,0.0,0.0,...,2.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
#Create the UMAP pipeline, first do unsupervised regular UMAP, then compare with supervised UMAP
import umap

%%time
embedding = umap.UMAP(n_neighbors=5).fit_transform(feature_df_target_disease)

UsageError: Line magic function `%%time` not found.


In [ ]:
#Plot the UMAP embeddings